In [1]:
# Install dependencies
!pip install nltk tensorflow scikit-learn --quiet

import nltk
from nltk.corpus import movie_reviews
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Download dataset
nltk.download('movie_reviews')

# Load and shuffle data
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)

# Prepare data
texts = [" ".join(words) for words, label in documents]
labels = [label for words, label in documents]

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)  # 0=neg, 1=pos

# Split data
X_train, X_test, y_train, y_test = train_test_split(texts, y, test_size=0.2, random_state=42)

# Text vectorization using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train).toarray()
X_test_vec = vectorizer.transform(X_test).toarray()

# Build Neural Network Model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_vec.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Output: sentiment (0 or 1)
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
print("\nTraining Neural Network...\n")
model.fit(X_train_vec, y_train, epochs=5, batch_size=32, validation_split=0.1, verbose=1)

# Evaluate
loss, acc = model.evaluate(X_test_vec, y_test)
print(f"\n Test Accuracy: {acc*100:.2f}%")

# Test with custom input
sample = ["The film was dull and boring. Not recommended."]
sample_vec = vectorizer.transform(sample).toarray()
prediction = model.predict(sample_vec)
sentiment = "Positive " if prediction[0][0] > 0.5 else "Negative "
print("\n Review:", sample[0])
print("Predicted Sentiment:", sentiment)

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Training Neural Network...

Epoch 1/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.5770 - loss: 0.6872 - val_accuracy: 0.7563 - val_loss: 0.6269
Epoch 2/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8958 - loss: 0.4876 - val_accuracy: 0.8313 - val_loss: 0.4461
Epoch 3/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9703 - loss: 0.1460 - val_accuracy: 0.7750 - val_loss: 0.4506
Epoch 4/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9966 - loss: 0.0436 - val_accuracy: 0.7937 - val_loss: 0.4668
Epoch 5/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 1.0000 - loss: 0.0136 - val_accuracy: 0.8062 - val_loss: 0.4928
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8479 - loss: 0.3446  

 Test Accuracy: 85.00%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step

 Review: The film was dull and boring. Not recommended.
Predicted Sentiment: Negative 
